### MarineTraffic Power Tool - Fleets (Modify Fleet)


API documentation https://servicedocs.marinetraffic.com/tag/Fleets

API key can be found at https://www.marinetraffic.com/en/users/my_account/api/account



In [ ]:
import time
import requests  # type: ignore
import xml.etree.ElementTree as ET
import pandas as pd # type: ignore
import os

In [ ]:
# Variables to adjust
api_key_getfleet = "apikey1"  # Get vessels in fleet
api_key_setfleet = "apikey2"  # Update/Add fleet URL
api_key_clearfleet = "apikey3"  # Clear fleet URL
api_key_listfleetids = "apikey4"  # Retrieve fleet IDs

# Editable variable for the file path
file_path = "E:\example-vessel-list.csv"  # Change this to your desired path

fleet_id = "2558323"
identifier = "imo"  # Can be "imo", "mmsi", or "ship_id"
# Example IMO numbers (or MMSI, or SHIP_ID depending on identifier)
VESSELS = [

    # Add more vessel numbers as needed
]

# API URL templates
get_fleet_url = f"https://services.marinetraffic.com/api/getfleet/{api_key_getfleet}/v:2/fleet_id:{fleet_id}/protocol:xml"  # Get vessels in fleet
set_fleet_url_template = f"https://services.marinetraffic.com/api/setfleet/{api_key_setfleet}/{identifier}"  # Update/Add fleet URL
clear_fleet_url_template = f"https://services.marinetraffic.com/api/clearfleet"  # Clear fleet URL
list_fleets_url = f"https://services.marinetraffic.com/api/getfleets/{api_key_listfleetids}/protocol:xml"  # List fleet IDs URL

In [ ]:
# Step 1: Fetch current vessels in the fleet
def fetch_vessels_in_fleet():
    response = requests.get(get_fleet_url)

    if response.status_code == 200:
        try:
            # Parse the XML response
            root = ET.fromstring(response.content)
            vessels_in_fleet = []

            # Loop through the VESSEL elements
            for vessel in root.findall('VESSEL'):
                vessel_data = {
                    "ship_id": vessel.get("SHIP_ID"),
                    "mmsi": vessel.get("MMSI"),
                    "imo": vessel.get("IMO"),
                    "shipname": vessel.get("SHIPNAME"),
                    "active": vessel.get("ACTIVE"),
                }
                vessels_in_fleet.append(vessel_data)

            return vessels_in_fleet

        except ET.ParseError as e:
            print(f"Failed to parse XML. Error: {e}.")
            return []
    else:
        print(f"Error fetching vessels: {response.status_code}.")
        return []

# Step 2: Identify vessels to add
def identify_vessels_to_add(vessels_in_fleet, vessels_to_add):
    # Get a set of identifiers already in the fleet based on the chosen identifier variable
    fleet_identifiers = {vessel[identifier] for vessel in vessels_in_fleet}  # Use the dynamic identifier

    # Identify vessels to add that are not in the fleet
    to_add = [vessel for vessel in vessels_to_add if vessel[identifier] not in fleet_identifiers]
    to_skip = [vessel for vessel in vessels_to_add if vessel[identifier] in fleet_identifiers]

    return to_add, to_skip

# Step 3: Add vessels to the fleet
def add_vessels_to_fleet(vessels_to_add):
    vessels_in_fleet = fetch_vessels_in_fleet()

    # Identify vessels that need to be added and those to be skipped
    vessels_to_add, vessels_to_skip = identify_vessels_to_add(vessels_in_fleet, vessels_to_add)

    # Notify how many vessels will be added and how many will be skipped
    print(f"Number of vessels to be added: {len(vessels_to_add)}")
    print(f"Number of vessels skipped (already in fleet): {len(vessels_to_skip)}")

    # List skipped vessels
    if vessels_to_skip:
        print("Skipped vessels (already in the fleet):")
        for vessel in vessels_to_skip:
            print(f"Vessel: {vessel['shipname']} (IMO: {vessel['imo']})")

    failed_vessels = []
    added_vessels = []

    # Add vessels to the fleet
    for vessel in vessels_to_add:
        vessel_number = vessel[identifier]  # Using the dynamic identifier (IMO, MMSI, etc.)
        set_fleet_url = f"{set_fleet_url_template}:{vessel_number}/fleet_id:{fleet_id}/active:1"
        response = requests.get(set_fleet_url)

        if response.status_code == 200:
            print(f"Vessel {vessel['shipname']} (IMO: {vessel['imo']}) added successfully.")
            added_vessels.append(vessel)
        else:
            print(f"Error adding vessel {vessel['shipname']} (IMO: {vessel['imo']}): {response.text}")
            failed_vessels.append(vessel)

        # Sleep between requests to avoid hitting rate limits
        time.sleep(5)

    # Summary of added vessels
    if added_vessels:
        print("Vessels successfully added to the fleet:")
        for vessel in added_vessels:
            print(f"Vessel: {vessel['shipname']} (IMO: {vessel['imo']})")

    # Summary of failed vessels
    if failed_vessels:
        print(f"The following vessels were not added due to errors:")
        for vessel in failed_vessels:
            print(f"Vessel: {vessel['shipname']} (IMO: {vessel['imo']})")

    return added_vessels, vessels_to_skip, failed_vessels

# Step 3.5: Load vessels from file or use the VESSELS array
def load_vessels_from_file():
    # If there are values in the VESSELS list, use them
    if VESSELS:
        print("Using predefined VESSELS list.")
        return VESSELS

    # If VESSELS list is empty, load vessels from the CSV file
    if not os.path.exists(file_path):
        print(f"File {file_path} does not exist and VESSELS list is empty.")
        return []

    try:
        df = pd.read_csv(file_path)
        
        # Check if the identifier column exists
        if identifier not in df.columns:
            print(f"Column '{identifier}' not found in the file.")
            return []
        
        # Filter out rows where the identifier column is empty or NaN
        valid_vessels = df[identifier].dropna().tolist()
        
        if not valid_vessels:
            print(f"No valid vessels found in the file.")
            return []

        return valid_vessels

    except Exception as e:
        print(f"Error reading vessels from file: {e}")
        return []

# Step 4: Remove vessels from the fleet
def remove_vessels_from_fleet(vessels_to_remove):
    vessels_in_fleet = fetch_vessels_in_fleet()
    fleet_identifiers = {vessel[identifier]: vessel for vessel in vessels_in_fleet}

    failed_vessels = []

    for vessel_number in vessels_to_remove:
        if vessel_number in fleet_identifiers:
            # Format the set fleet URL with the correct vessel number and delete parameter
            set_fleet_url = f"{set_fleet_url_template}:{vessel_number}/fleet_id:{fleet_id}/delete:1"
            response = requests.get(set_fleet_url)

            if response.status_code == 200:
                print(f"Vessel {vessel_number} removed successfully.")
            else:
                print(f"Error removing vessel {vessel_number}: {response.text}")
                failed_vessels.append(vessel_number)
        else:
            print(f"Vessel {vessel_number} is not in the fleet.")

        # Step 4: Add a 5-second buffer between API calls to avoid overwhelming the API
        time.sleep(5)

    if failed_vessels:
        print(f"The following vessels could not be removed due to errors: {failed_vessels}")

# Step 5: Set vessels active or inactive
def set_vessel_active(vessels_to_set_active):
    for vessel_number in vessels_to_set_active:
        # Format the set fleet URL with the correct vessel number and active parameter
        set_fleet_url = f"{set_fleet_url_template}:{vessel_number}/fleet_id:{fleet_id}/active:1"
        response = requests.get(set_fleet_url)

        if response.status_code == 200:
            print(f"Vessel {vessel_number} set to active successfully.")
        else:
            print(f"Error setting vessel {vessel_number} to active: {response.text}")

        # Step 4: Add a 5-second buffer between API calls to avoid overwhelming the API
        time.sleep(5)

def set_vessel_inactive(vessels_to_set_inactive):
    for vessel_number in vessels_to_set_inactive:
        # Format the set fleet URL with the correct vessel number and inactive parameter
        set_fleet_url = f"{set_fleet_url_template}:{vessel_number}/fleet_id:{fleet_id}/active:0"
        response = requests.get(set_fleet_url)

        if response.status_code == 200:
            print(f"Vessel {vessel_number} set to inactive successfully.")
        else:
            print(f"Error setting vessel {vessel_number} to inactive: {response.text}")

        # Step 4: Add a 5-second buffer between API calls to avoid overwhelming the API
        time.sleep(5)

# Step 6: List vessels in fleet using XML protocol
def list_vessels_in_fleet():
    response = requests.get(get_fleet_url)

    if response.status_code == 200:
        # Parse the XML response
        root = ET.fromstring(response.content)
        vessels_in_fleet = []

        # Loop through the VESSEL elements
        for vessel in root.findall('VESSEL'):
            vessel_data = {
                "ship_id": vessel.get("SHIP_ID"),
                "mmsi": vessel.get("MMSI"),
                "imo": vessel.get("IMO"),
                "shipname": vessel.get("SHIPNAME"),
                "active": vessel.get("ACTIVE"),
            }
            vessels_in_fleet.append(vessel_data)
        
        return vessels_in_fleet
    else:
        print(f"Error fetching vessels: {response.status_code}.")
        return []

# Step 7: Clear all vessels in the fleet
def clear_vessels_in_fleet():
    confirmation = input("Are you sure you want to remove all vessels in the fleet? (Y/N): ").strip().upper()
    
    if confirmation == 'Y':
        clear_fleet_url = f"{clear_fleet_url_template}/{api_key_clearfleet}/fleet_id:{fleet_id}"
        response = requests.get(clear_fleet_url)
        
        if response.status_code == 200:
            print("All vessels in the fleet have been cleared successfully.")
        else:
            print(f"Error clearing fleet: {response.status_code}. Response: {response.text}")
    elif confirmation == 'N':
        print("Fleet clearance operation canceled.")
    else:
        print("Invalid option. Please enter Y or N.")

# Step 8: List fleet ids using XML protocol
def list_fleet_ids():
    get_fleet_url = f"https://services.marinetraffic.com/api/getfleets/{api_key_listfleetids}/protocol:xml" 
    response = requests.get(get_fleet_url)

    if response.status_code == 200:
        try:
            # Parse the XML response
            root = ET.fromstring(response.content)

            # Find all FLEET elements
            fleets = root.findall('FLEET')

            if fleets:
                print("Fleets found:")
                for fleet in fleets:
                    fleet_info = {
                        "ID": fleet.get("ID", "N/A"),
                        "NAME": fleet.get("NAME", "N/A"),
                        "ACTIVE": fleet.get("ACTIVE", "N/A"),
                        "CREATED": fleet.get("CREATED", "N/A"),
                        "DEFAULT": fleet.get("DEFAULT", "N/A")
                    }
                    print(fleet_info)
            else:
                print("No fleets found.")

        except ET.ParseError:
            print("Failed to parse XML. Response content is not in valid XML format.")
    else:
        print(f"Error fetching fleets: {response.status_code}")

In [ ]:
# Main script to prompt user for action
def main():
    while True:
        print("\nChoose an action:")
        print("1. Add vessels")
        print("2. Remove vessels")
        print("3. Set vessels active")
        print("4. Set vessels inactive")
        print("5. List vessels in fleet")
        print("6. Clear all vessels in fleet")   
        print("7. List Fleet IDs")        
        print("8. Exit")

        choice = input("Enter your choice (1-8): ")

        if choice == "1":
            add_vessels_to_fleet(load_vessels_from_file())
        elif choice == "2":
            vessels_to_remove = VESSELS if VESSELS else input("Enter vessel identifiers to remove (comma-separated): ").split(',')
            remove_vessels_from_fleet(vessels_to_remove)
        elif choice == "3":
            vessels_to_set_active = VESSELS if VESSELS else input("Enter vessel identifiers to set active (comma-separated): ").split(',')
            set_vessel_active(vessels_to_set_active)
        elif choice == "4":
            vessels_to_set_inactive = VESSELS if VESSELS else input("Enter vessel identifiers to set inactive (comma-separated): ").split(',')
            set_vessel_inactive(vessels_to_set_inactive)
        elif choice == "5":
            vessels_in_fleet = list_vessels_in_fleet()
            for vessel in vessels_in_fleet:
                print(vessel)
        elif choice == '6':
            clear_vessels_in_fleet()
        elif choice == '7':
            list_fleet_ids()
        elif choice == "8":
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please enter a number between 1 and 8.")

if __name__ == "__main__":
    main()